In [ ]:
# 구글맵에서 vegan restaurants in Paris로 검색한 결과 전체 레스토랑정보 수집 output/google_result.json 결과 저장
# https://serpapi.com/
import requests
import json
import os


def get_vegan_restaurants_and_reviews(api_key, search_query="Vegan restaurants in Paris", location="Paris, France", max_reviews=5):
    """
    Searches for all vegan-friendly restaurants in Paris and retrieves information,
    including name, rating, review count, address, Google Maps link, and up to 5 reviews per restaurant.

    :param api_key: SerpApi API key
    :param search_query: Search term (default is 'Vegan restaurants in Paris')
    :param location: Search location (default is 'Paris, France')
    :param max_reviews: Maximum number of reviews to fetch for each restaurant
    """
    results = []
    start_index = 0
    page_results_limit = 20  # Number of results per page (SerpApi default limit)
    ll = "@48.8566,2.3522,14z"

    while True:
        # Set up SerpApi search parameters
        params = {
            "engine": "google_maps",  # Google Maps API engine
            "q": search_query,       # Search term
            "location": location,    # Search location
            "type": "search",        # Search type
            "start": start_index,    # Pagination start
            "ll": ll,                # Latitude, Longitude of Paris
            "api_key": api_key       # API key
        }

        # Send request to SerpApi
        response = requests.get("https://serpapi.com/search", params=params)
        
        if response.status_code != 200:
            print(f"Error: {response.status_code}, {response.text}")
            break

        # Parse JSON data
        data = response.json()
        local_results = data.get("local_results", [])

        # If no more results, stop the loop
        if not local_results:
            break

        # Process each restaurant
        for restaurant in local_results:
            name = restaurant.get("title", "No information available")
            address = restaurant.get("address", "No address available")
            rating = restaurant.get("rating", "No rating available")
            reviews_count = restaurant.get("reviews", "No reviews available")
            google_maps_url = restaurant.get("gps_coordinates_link", "No link available")
            data_id = restaurant.get("data_id")  # Unique identifier required for fetching reviews

            print(f"Restaurant: {name} | Rating: {rating} | Reviews: {reviews_count} | Address: {address}")
            print(f"Google Maps URL: {google_maps_url}")
            print("-" * 50)

            # Prepare restaurant entry
            restaurant_entry = {
                "name": name,
                "address": address,
                "rating": rating,
                "reviews_count": reviews_count,
                "google_maps_url": google_maps_url,
                "reviews": []
            }

            # Fetch reviews if data_id is available
            if data_id:
                reviews = get_reviews_for_restaurant(data_id, api_key, max_reviews)
                restaurant_entry["reviews"] = reviews

            # Add to results list
            results.append(restaurant_entry)

        # Move to the next page of results
        start_index += page_results_limit

    # Save results to JSON file
    save_results_to_file(results, "output/google_result.json")

    # Output total restaurant count
    print(f"\nTotal restaurants found: {len(results)}")


def get_reviews_for_restaurant(data_id, api_key, max_reviews):
    """
    Fetches review data for a specific restaurant.

    :param data_id: Unique identifier of the restaurant
    :param api_key: SerpApi API key
    :param max_reviews: Maximum number of reviews to fetch
    :return: List of reviews
    """
    # Set up SerpApi review request parameters
    params = {
        "engine": "google_maps_reviews",  # Review engine
        "data_id": data_id,              # Unique identifier of the restaurant
        "api_key": api_key               # API key
    }

    # Send request to SerpApi
    response = requests.get("https://serpapi.com/search", params=params)
    
    if response.status_code != 200:
        print(f"Error fetching review data. Status code: {response.status_code}")
        return []

    # Parse review data
    reviews_data = response.json().get("reviews", [])
    reviews = []
    
    for review in reviews_data[:max_reviews]:
        review_text = review.get("snippet", "No review text available")
        reviews.append(review_text)

    return reviews


def save_results_to_file(results, file_path):
    """
    Saves the given results to a JSON file.

    :param results: Data to save
    :param file_path: Path to the output file
    """
    # Ensure the output directory exists
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    # Write JSON data to the file
    with open(file_path, "w", encoding="utf-8") as json_file:
        json.dump(results, json_file, ensure_ascii=False, indent=4)

    print(f"Results saved to {file_path}")


# Run the function
if __name__ == "__main__":
    API_KEY = "709d54dffa8e2a67086779a6edb35e105edfc87fde59e348fe6de55a8bd696d9"  # Insert your SerpApi API key here
    SEARCH_QUERY = "Vegan restaurants in Paris"  # Search keyword
    LOCATION = "Paris, France"                   # Search location
    MAX_REVIEWS = 5                              # Number of reviews to fetch per restaurant

    get_vegan_restaurants_and_reviews(API_KEY, SEARCH_QUERY, LOCATION, MAX_REVIEWS)



Restaurant: Le Potager du Marais | Rating: 4.3 | Reviews: 1585 | Address: 26 Rue Saint-Paul, 75004 Paris, France
Google Maps URL: No link available
--------------------------------------------------
Restaurant: Jah Jah By Le Tricycle | Rating: 4.7 | Reviews: 1625 | Address: 11 R. des Petites Écuries, 75010 Paris, France
Google Maps URL: No link available
--------------------------------------------------
Restaurant: Le Potager de Charlotte | Rating: 4.7 | Reviews: 1917 | Address: 12 Rue Louise-Émilie de la Tour d'Auvergne, 75009 Paris, France
Google Maps URL: No link available
--------------------------------------------------
Restaurant: Aujourd'hui Demain | Rating: 4.7 | Reviews: 2920 | Address: 42 Rue du Chemin Vert, 75011 Paris, France
Google Maps URL: No link available
--------------------------------------------------
Restaurant: Green Farmer’s | Rating: 4.8 | Reviews: 607 | Address: 49 Bd de Clichy, 75009 Paris, France
Google Maps URL: No link available
-------------------------

In [ ]:
# 1 try
import requests

def get_google_maps_reviews():
    api_key = "709d54dffa8e2a67086779a6edb35e105edfc87fde59e348fe6de55a8bd696d9"

    params = {
        "engine": "google_maps",
        "q": "Paris vegan",
        "location": "France, Paris",
        "type": "search",
        "api_key": api_key,
    }

    response = requests.get("https://serpapi.com/search", params=params)

    if response.status_code != 200:
        print("Error:", response.json())
        return

    data = response.json()
    restaurants = data.get("local_results", [])

    for restaurant in restaurants:
        name = restaurant.get("title")
        address = restaurant.get("address")
        rating = restaurant.get("rating")
        data_id = restaurant.get("data_id")  # data_id를 가져옴

        print(f"레스토랑 이름: {name}")
        print(f"주소: {address}")
        print(f"평점: {rating}")
        print("-" * 50)

        if data_id:  # data_id로 리뷰 데이터를 가져옴
            fetch_reviews(data_id, api_key)
        else:
            print(f"리뷰 데이터를 가져올 수 없습니다. 레스토랑 이름: {name}")

def fetch_reviews(data_id, api_key):
    params = {
        "engine": "google_maps_reviews",
        "data_id": data_id,  # `data_id`를 활용
        "api_key": api_key,
    }

    response = requests.get("https://serpapi.com/search", params=params)

    if response.status_code != 200:
        print("리뷰 데이터를 가져오는 데 오류가 발생했습니다.")
        return

    reviews_data = response.json().get("reviews", [])
    for review in reviews_data:
        reviewer = review.get("user")
        rating = review.get("rating")
        text = review.get("snippet")

        print(f"리뷰어: {reviewer}")
        print(f"평점: {rating}")
        print(f"리뷰 내용: {text}")
        print("-" * 30)

# 함수 실행
get_google_maps_reviews()


레스토랑 이름: Aujourd'hui Demain
주소: 42 Rue du Chemin Vert, 75011 Paris, France
평점: 4.7
--------------------------------------------------
리뷰어: {'name': 'Lucas Poyser', 'link': 'https://www.google.com/maps/contrib/112260745565742015308?hl=en-US', 'contributor_id': '112260745565742015308', 'thumbnail': 'https://lh3.googleusercontent.com/a-/ALV-UjVE3pVy0-FnhcbD9WgSojTWlxsY59a1OqTz2giR5DWf1bo85WqC=s120-c-rp-mo-ba4-br100', 'local_guide': True, 'reviews': 93, 'photos': 159}
평점: 5.0
리뷰 내용: As far as vegan restaurants go in Paris, this is the BEST one! We went twice in a week because it was so good. There is a also really cute thrift shop in the back of the main seating area. Outdoor seating is available. Be aware that there can be a short wait for lunch unless you are a pair or small group. They have amazing lunch specials as well that you can take advantage of.
------------------------------
리뷰어: {'name': 'Avery', 'link': 'https://www.google.com/maps/contrib/110967355260619352473?hl=en-US', 'con